# Data Envelopment Analysis BCC-O
The following functions allow to perform BCC-O DEA analysis

In [1]:
from scipy.optimize import linprog
import pandas as pd
import os

from sqlalchemy import create_engine, text
from decimal import *

### Definitions for SQL Connection

In [2]:
#maximum number of rows to display
pd.options.display.max_rows = 20

DB_USERNAME = 'alagos'
DB_PASSWORD = 'Team67!'
DB_ENDPOINT = 'ds4a-demo-instance.cqjr4hyu9xaq.us-east-1.rds.amazonaws.com'
DB_NAME = 'desertion_pj_team67'
engine=create_engine(f'postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_ENDPOINT}/{DB_NAME}', max_overflow=20)

def runQuery(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

Some test data

In [3]:
df = runQuery("""
select * from dane.var_dpto;""")

In [4]:
data = df[['codigointernodepto','alu_06', 'alu_07', 'alu_08']]
data = data.rename(columns={'codigointernodepto': 'DMU'})
data['DMU'] = data['DMU'].apply(str)

# Define the input and output variables
inp = ['alu_06', 'alu_07']
out = ['alu_08']

### Definitions for Phase I

In [5]:
# Set base model for Phase I
def BCCO_Base_PH1(inp, out):
    # Initialize components of LP
    global obj1, lhs_eq1, rhs_eq1, lhs_ineq1, rhs_ineq1, bnd1
    obj1, lhs_eq1, rhs_eq1, lhs_ineq1, rhs_ineq1, bnd1 = ([] for i in range(6))
    
    # Set linear inequalities
    tmp = []
    for i in inp:
        tmp = data[i].tolist()
        tmp.insert(0,0)
        lhs_ineq1.append(tmp)
    for o in out:
        tmp = data[o].tolist()
        tmp = [-1 * i for i in tmp]
        tmp.insert(0,0)
        lhs_ineq1.append(tmp)
    
    obj1 = [-1]
    tmp = [0]
    rhs_eq1 = [1]
    bnd1 = [(float("-inf"), float("inf"))]
    for k in range(len(data[inp[0]])):
        obj1.append(0)
        tmp.append(1)
        bnd1.append((0, float("inf")))
    lhs_eq1.append(tmp)

# Set model for each DMU for Phase I
def BCCO_DMU_PH1(dmu, inp, out):
    # Initialize components of LP
    global rhs_ineq1
    rhs_ineq1 = []
    index = 0
    
    # Data from selected DMU
    dmu_data = data[data['DMU'] == dmu].reset_index()
    
    for i in inp:
        lhs_ineq1[index][0] = 0
        rhs_ineq1.append(dmu_data[i][0])
        index = index +1
    
    for o in out:
        lhs_ineq1[index][0] = dmu_data[o][0]
        rhs_ineq1.append(0)
        index = index + 1

def BCCO_DMU_VAR(inp, out, slack):
    varset = []
    nInp = len(inp)
    for i in range(len(slack)):
        if slack[i] > 0.01:
            if i < len(inp):
                varset.append(inp[i])
            else:
                varset.append(out[i-nInp])
    return varset

### Definitions for Phase II

In [6]:
# Set base model for Phase II
def BCCO_Base_PH2(inp, out):
    # Initialize components of LP
    global obj2, lhs_eq2, rhs_eq2, bnd2
    obj2, lhs_eq2, rhs_eq2, bnd2 = ([] for i in range(4))
    
    # Set linear inequalities
    nSlk = len(inp)+len(out)
    tmp = []
    index = 0
    
    for i in inp:
        tmp = data[i].tolist()
        for j in range(nSlk):
            tmp.insert(0,0)
        tmp[index] = 1
        lhs_eq2.append(tmp)
        index = index + 1

    for o in out:
        tmp = data[o].tolist()
        #tmp = [-1 * i for i in tmp]
        for j in range(nSlk):
            tmp.insert(0,0)
        tmp[index] = -1
        lhs_eq2.append(tmp)
        index = index + 1
    
    tmp = []
    for j in range(nSlk):
        obj2.append(1)
        bnd2.append((0, float("inf")))
        tmp.append(0)
    
    for k in range(len(data[inp[0]])):
        obj2.append(0)
        bnd2.append((0, float("inf")))
        tmp.append(1)
    lhs_eq2.append(tmp)
        
# Set model for each DMU for Phase II
def BCCO_DMU_PH2(dmu, theta, inp, out):
    # Initialize components of LP
    global rhs_eq2
    rhs_eq2 = []
    
    # Data from selected DMU
    dmu_data = data[data['DMU'] == dmu].reset_index()
    
    for i in inp:
        rhs_eq2.append(dmu_data[i][0])
    
    for o in out:
        rhs_eq2.append(theta*dmu_data[o][0])
    
    rhs_eq2.append(1)

# Determine which DMUs comprise the reference set.
def BCCO_DMU_REFSET(inp, out, ph2_x):
    dmu_names = data['DMU'].reset_index()
    nSlk = len(inp)+len(out)
    res = ph2_x[nSlk:]
    refset = []
    for i in range(len(res)):
        if res[i] > 0:
            refset.append(dmu_names.iloc[i][1])
    return refset

Using the DEA functions

In [7]:
# Load the models for Phase I and Phase II
BCCO_Base_PH1(inp, out)
BCCO_Base_PH2(inp, out)

In [8]:
# Solves for all DMUs
for dmu in data['DMU'].tolist():
    BCCO_DMU_PH1(dmu, inp, out)
    ph1_dual = linprog(c=obj1, A_ub=lhs_ineq1, b_ub=rhs_ineq1, A_eq=lhs_eq1, b_eq=rhs_eq1, bounds=bnd1, 
                       method="simplex")
    print(dmu + ': ' + str(-1/ph1_dual.fun), end = ' --> ')
    BCCO_DMU_PH2(dmu, -1*Decimal(ph1_dual.fun), inp, out)
    ph2 = linprog(c=obj2, A_eq=lhs_eq2, b_eq=rhs_eq2, bounds=bnd2,method="simplex")
    print(BCCO_DMU_REFSET(inp,out,ph2.x), end = ' --> ')
    print(BCCO_DMU_VAR(inp, out, ph1_dual.slack))

99: 0.7965991795622868 --> ['97', '44', '27', '5'] --> []
54: 0.07738030932245504 --> ['44', '19', '5'] --> ['alu_07']
88: 0.2393162393162393 --> ['97'] --> ['alu_06']
68: 0.07331189710610939 --> ['19', '5'] --> ['alu_06', 'alu_07']
41: 0.15218819705012737 --> ['44', '19', '5'] --> ['alu_07']
70: 0.20147694512618 --> ['44', '19', '5'] --> ['alu_07']
52: 0.8951768488745981 --> ['19', '5'] --> ['alu_06', 'alu_07']
63: 0.06209704859991417 --> ['44', '19', '5'] --> ['alu_07']
15: 0.0443729903536978 --> ['19', '5'] --> ['alu_06', 'alu_07']
86: 0.3716152304609218 --> ['44', '27'] --> ['alu_07']
50: 0.22815397799124637 --> ['44', '19'] --> ['alu_07']
97: 1.0 --> ['97'] --> []
47: 0.16411651428287657 --> ['44', '19', '5'] --> ['alu_07']
66: 0.2710034407174856 --> ['44', '19'] --> ['alu_07']
91: 0.5819338610638455 --> ['97', '44'] --> ['alu_06']
81: 0.2075969576690247 --> ['97', '27', '95'] --> ['alu_07']
17: 0.18804822075850117 --> ['44', '19', '5'] --> ['alu_07']
20: 0.3640344446294093 --> ['